In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import mediapipeFuncions
from tensorflow.keras.utils import to_categorical
mp_drawing = mp.solutions.drawing_utils          # mediapipe 繪圖方法
mp_drawing_styles = mp.solutions.drawing_styles  # mediapipe 繪圖樣式
mp_pose = mp.solutions.pose                      # mediapipe 姿勢偵測
mp_holistic = mp.solutions.holistic # Holistic model

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]='-1' 

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_flip') 

# Actions that we try to detect
actions = np.array(['Reading', 'Writing', 'PlayWithPhone'])

# Thirty videos worth of data
no_sequences = 3

# Videos are going to be 30 frames in length
sequence_length = 60

# Folder start
start_folder = 0

In [ ]:
for action in actions: # This will loop through all the different gestures
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence))) # This will create all necesary folders, subfolders and sequences
        except:
            pass

In [ ]:
# Extracting skeleton points for Level3 training data using Mediapipe.
# Reading 
video_reading  = np.array(['training/3/S001C001P001R001A011_rgb.avi',
                           'training/3/S001C001P003R001A011_rgb.avi',
                           'training/3/S001C001P003R002A011_rgb.avi',])
# Writing
video_writing  = np.array(['training/3/S001C001P001R001A012_rgb.avi',
                           'training/3/S001C001P003R001A012_rgb.avi',
                           'training/3/S001C001P003R002A012_rgb.avi',])
# Play with phone
video_play = np.array(['training/3/S001C001P001R001A029_rgb.avi',
                           'training/3/S001C001P003R001A029_rgb.avi',
                           'training/3/S001C001P003R002A029_rgb.avi',])
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
   
    # NEW LOOP
    # Loop through actions
     for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            if(action == 'Reading'):
                cap = cv2.VideoCapture(video_reading[sequence])
            elif(action == 'Writing'):
                cap = cv2.VideoCapture(video_writing[sequence])
            elif(action == 'PlayWithPhone'):
                cap = cv2.VideoCapture(video_play[sequence])
            for frame_num in range(sequence_length):
                
                # Read feed
                ret, frame = cap.read()
                frame_cut = frame[135:850, 640:1440]
                frame_cut = cv2.flip(frame_cut, 1)
                M = cv2.getRotationMatrix2D(((800-1)/2.0, (715-1/2.0)), 5, 1)
                frame_cut = cv2.warpAffine(frame_cut, M, (800,715))
                
                mat_translation = np.float32([[1, 0, -50], [0, 1, -50]]) # 變換矩陣：設定平移變換所需的計算矩陣：2行3列
                frame_cut = cv2.warpAffine(frame_cut, mat_translation, (800 - 50, 715 - 50))  # 變換函數
                # Make detections
                image, results = mediapipeFuncions.mediapipe_detection(frame_cut, holistic)

                # Draw landmarks
                mediapipeFuncions.draw_styled_landmarks(image, results)
#                 print(frame_num)
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = mediapipeFuncions.extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 os.makedirs(npy_path)
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q') or frame_num == 59:
                    break

     cap.release()
     cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
DATA_PATH = os.path.join('MP_Data_flip') 

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)
X.shape

In [ ]:
y = to_categorical(labels).astype(int)
y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
from datetime import datetime
# log_dir = os.path.join('Logs')
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
from keras.layers import Dropout
from keras import regularizers
model = Sequential()
# model.add(LSTM(32, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=</ACV_results>

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X, y, epochs=1500,batch_size=6, callbacks=[tb_callback])

In [ ]:
#model.save('my_model.h5')